### __Resnet vs Densenet Assignment | Mohammed Asif Sahadh - 24MSD7061__

#### Load Resnet model

In [ ]:
import torch
import torchvision.models as models

model = models.resnet18(pretrained = True)

#### Change output layer

In [5]:
num_classes = 5
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

#### Data loading & processing

In [7]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

train_dataset = ImageFolder('face/train', transform = transform)
val_dataset = ImageFolder('face/val', transform = transform)
test_dataset = ImageFolder('face/test', transform = transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

#### Define loss function & optimizer

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr = 0.01, momentum = 0.9)

#### Define training loop

In [11]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(num_epochs):

        print(f"Epoch {epoch + 1}...")
        # set the model to training mode
        model.train()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # reset the gradients to zero before the backward pass
            optimizer.zero_grad()

            # forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            # backward pass: compute gradients
            loss.backward()

            # update model parameters
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects.float() / len(train_loader.dataset)

        # set the model to evaluation mode for validation
        model.eval()

        running_loss = 0.0
        running_corrects = 0

        # disable gradient computation for validation (saves memory and computations)
        with torch.no_grad():

            for inputs, labels in val_loader:

                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward pass: compute the model output
                outputs = model(inputs)
                # get the predicted class (with the highest score)
                _, preds = torch.max(outputs, 1)
                # compute the loss between the predictions and actual labels
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        val_loss = running_loss / len(val_loader.dataset)
        val_acc = running_corrects.float() / len(val_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}')

#### Model training

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs = 5)

Epoch 1...
Epoch [1/5], train loss: 1.3128, train acc: 0.5381, val loss: 2.0901, val acc: 0.4042
Epoch 2...
Epoch [2/5], train loss: 1.2426, train acc: 0.5582, val loss: 2.9313, val acc: 0.3521
Epoch 3...
Epoch [3/5], train loss: 1.2661, train acc: 0.5592, val loss: 2.6361, val acc: 0.3604
Epoch 4...
Epoch [4/5], train loss: 1.2846, train acc: 0.5620, val loss: 2.0494, val acc: 0.4271
Epoch 5...
Epoch [5/5], train loss: 1.2571, train acc: 0.5716, val loss: 2.1428, val acc: 0.4083


#### Repeat the same steps for Densenet 

In [15]:
model = models.densenet121(pretrained = True)

In [18]:
num_classes = 5
model.fc = torch.nn.Linear(model.classifier.in_features, num_classes)

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier.parameters(), lr = 0.01, momentum = 0.9)

In [23]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(num_epochs):

        print(f"Epoch {epoch + 1}...")
        # set the model to training mode
        model.train()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # reset the gradients to zero before the backward pass
            optimizer.zero_grad()

            # forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            # backward pass: compute gradients
            loss.backward()

            # update model parameters
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects.float() / len(train_loader.dataset)

        # set the model to evaluation mode for validation
        model.eval()

        running_loss = 0.0
        running_corrects = 0

        # disable gradient computation for validation (saves memory and computations)
        with torch.no_grad():

            for inputs, labels in val_loader:

                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward pass: compute the model output
                outputs = model(inputs)
                # get the predicted class (with the highest score)
                _, preds = torch.max(outputs, 1)
                # compute the loss between the predictions and actual labels
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        val_loss = running_loss / len(val_loader.dataset)
        val_acc = running_corrects.float() / len(val_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}')

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
train(model, train_loader, val_loader, criterion, optimizer, num_epochs = 5)

Epoch 1...
Epoch [1/5], train loss: 1.5069, train acc: 0.5029, val loss: 2.3069, val acc: 0.3771
Epoch 2...
Epoch [2/5], train loss: 1.2712, train acc: 0.5590, val loss: 2.1925, val acc: 0.4354
Epoch 3...
Epoch [3/5], train loss: 1.2629, train acc: 0.5696, val loss: 1.7259, val acc: 0.4625
Epoch 4...
Epoch [4/5], train loss: 1.2126, train acc: 0.5792, val loss: 1.9870, val acc: 0.4479
Epoch 5...
Epoch [5/5], train loss: 1.3152, train acc: 0.5726, val loss: 2.3747, val acc: 0.4021
